In [78]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, log_loss
import operator
import json
from IPython import display
import os
import warnings
import re

np.random.seed(0)
warnings.filterwarnings("ignore")
THRESHOLD = 4
import matplotlib
%matplotlib inline

In [79]:
Phones1 = pd.read_csv("Phones_Refined.csv")

In [80]:
Phones1.describe()

,brand,network_technology,GPRS,EDGE,dimentions,weight_g,SIM,display_resolution,OS,CPU,...,RAM,primary_camera,secondary_camera,loud_speaker,audio_jack,GPS,NFC,radio,battery,approx_price_EUR
count,5006,5006,5001,4993,5007,4519,5003,4927,3564,3522,...,3357,4725,4711,5005,4979,4991,747,4947,5005,5006
unique,77,35,41,43,4006,760,70,2314,481,475,...,188,600,334,18,6,85,48,86,829,112
top,Samsung,GSM / HSPA,Yes,Yes,-,130,Mini-SIM,5.0 inches,Android 4.4.2 (KitKat),Quad-core 1.2 GHz Cortex-A7,...,1 GB RAM,2 MP,No,Yes,Yes,Yes with A-GPS,Yes,FM radio,Removable Li-Ion 1500 mAh battery,70
freq,872,1864,2399,2326,170,100,2176,29,290,219,...,1014,539,1772,4289,3677,1956,629,1597,250,302


In [81]:
a = {}
a = set(Phones1['memory_card'])
print(a)

{nan, 'SD', 'SD  up to 2 GB', 'microSD  up to 128 GB (dedicated slot)', 'microSD  up to 32 GB (dedicated slot) - market dependent', 'microSD  up to 32 GB (dedicated slot)  1 GB included', 'microSD  up to 8 GB (dedicated slot)  4 GB included', 'Memory Stick Micro (M2)  up to 4 GB  256 MB included', 'microSD  up to 256 GB (dedicated slot)', 'microSD  up to 8 GB (dedicated slot)  1 GB included', 'microSD  up to 8 GB (dedicated slot)  128 MB included', 'SDIO/MMC  up to 2 GB', 'Memory Stick Micro (M2)  up to  2 GB  256 MB included', 'MiniSD', 'microSD  up to 32 GB (dedicated slot) - optional', 'microSD  up to 32 GB (dedicated slot)  8 GB included', 'Memory Stick Micro (M2)   up to 8 GB  512 MB included', 'microSD (dedicated slot)  1 GB included', 'Memory Stick Duo Pro  up to 4 GB', 'microSD  up to 256 GB (dedicated slot) - single-SIM model', 'Memory Stick Micro (M2)  up to 4 GB  1 GB included', 'microSD  up to 8 GB', 'microSD  up to 4 GB (dedicated slot)', 'microSD  up to 32 GB (dedicated s

In [82]:
Phones1['2G'] = Phones1['network_technology']
Phones1['3G'] = Phones1['network_technology']
Phones1['4G'] = Phones1['network_technology']

In [83]:
d = {}
d = set(Phones1['network_technology'])
print(d)

{nan, 'GSM / UMTS', 'No cellular connectivity', 'GSM / HSPA / LTE', 'HSPA / EVDO', 'GSM / CDMA / HSPA / EVDO / LTE', 'GSM / CDMA / HSPA / LTE', 'CDMA / CDMA2000', 'GSM / UMTS / HSPA / LTE', 'GSM / CDMA2000', 'CDMA / HSPA / LTE', 'GSM / HSPA / EVDO / LTE', 'GSM / HSPA / EVDO', 'GSM / CDMA / EVDO', 'GSM / CDMA / UMTS / EVDO', 'CDMA / LTE', 'GSM / UMTS / HSPA', 'CDMA / HSPA / EVDO', 'CDMA / EVDO / LTE', 'LTE', 'GSM / HSPA / CDMA2000 / LTE', 'GSM / CDMA / EVDO / LTE', 'GSM', 'GSM / HSPA / CDMA2000', 'CDMA / EVDO', 'CDMA', 'CDMA / HSPA', 'GSM / CDMA / HSPA / CDMA2000 / LTE', 'GSM / CDMA', 'HSPA / LTE', 'GSM / LTE', 'GSM / CDMA / HSPA / EVDO', 'GSM / CDMA / HSPA', 'CDMA / HSPA / EVDO / LTE', 'GSM / HSPA', 'HSPA'}


In [84]:
Phones2 = Phones1.replace({'memory_card': {'microSD (dedicated slot)': 'Yes', 'microSD  up to 128 GB (dedicated slot)': 'Yes', 'microSD  up to 512 GB (dedicated slot)': 'Yes', 'microSD  up to 256 GB (dedicated slot) - single-SIM model': 'Yes', 'miniSD': 'No', 'miniSD  miniSDHC': 'No', 'miniSD  128 MB included': 'No', 'miniSD  up to 8 GB': 'No', 'microSD  up to 128 GB (dedicated slot)': 'Yes', 'Memory Stick Micro (M2)  up to 16 GB': 'No', 'microSD  up to 64 GB (dedicated slot)': 'Yes', 'Memory Stick Micro (M2)  up to 16 GB  1 GB included': 'No', 'microSD (dedicated slot)  128 MB included': 'Yes', 'microSD  up to 64 GB (dedicated slot)/ 32 GB (SGP351)': 'Yes', 'Memory Stick Micro (M2)  up to 2 GB  64 MB included': 'No', 'microSD  up to 2 GB (dedicated slot)  1 GB included': 'Yes', 'RS-DV-MMC  64 MB included': 'Yes', 'microSD  up to 32 GB (uses SIM 2 slot)': 'Yes', 'MMC  up to 2 GB': 'Yes', 'miniSD up to 8 GB': 'Yes', 'microSD  up to 2 GB (dedicated slot)  256 MB included': 'Yes', 'Memory Stick Micro (M2)  up to 8 GB  8 GB included': 'No', 'SD  up to 1 GB': 'Yes', 'SDIO/MMC': 'Yes', 'microSD  up to 64 GB (uses SIM 2 slot)': 'Yes', 'To be confirmed': 'No', 'miniSD  up to 2 GB': 'No', 'microSD  up to 4 GB (dedicated slot)  1 GB included': 'Yes', 'microSD  up to 32 GB (dedicated slot) - optional': 'Yes', 'SDIO/MMC  up to 2 GB': 'Yes', 'microSD (dedicated slot)': 'Yes', 'microSD  up to 16 GB (dedicated slot)  1 GB included': 'Yes', 'miniSD  64 MB included': 'No', 'miniSD  1 GB included': 'Yes', 'microSD  up to 8 GB': 'Yes', 'microSD  up to 32 GB (dedicated slot)  1 GB included': 'Yes', 'microSD  up to 32/256 GB (dedicated slot)': 'Yes', 'RS-DV-MMC': 'Yes', 'miniSD  up to 32 GB': 'Yes', 'microSD  up to 64 GB (dedicated slot) - single-SIM model': 'Yes', 'Memory Stick Micro (M2)  up to 2 GB  512 MB included': 'No', 'Memory Stick Micro (M2)  up to 4 GB  64 MB included': 'No', 'microSD  up to 2 GB (dedicated slot)  128 MB included': 'Yes', 'SD/microSD  up to 32 GB (dedicated slot)': 'Yes', 'microSD  up to 32 GB (dedicated slot)  8 GB included': 'Yes', 'microSD (dedicated slot)  1 GB included': 'Yes', 'Memory Stick Micro (M2)  up to 16 GB  8 GB included': 'No', 'microSD  up to 32 GB (dedicated slot) - single-SIM model': 'Yes', 'Memory Stick Micro (M2)  up to 2 GB  128 MB included': 'No', 'microSD  up to 32 GB (dedicated slot)  4 GB included': 'Yes', '2 x SDIO': 'No', 'microSD  up to 64 GB': 'Yes', 'Memory Stick Micro (M2)  up to 4 GB  2 GB included': 'No', 'SD/MMC  SDIO': 'No', 'microSD  up to 32 GB (dedicated slot) - not user accessible': 'No', 'Memory Stick Micro (M2)': 'No', 'SD': 'Yes', 'microSD  up to 16 GB (dedicated slot)  512 MB/ 1 GB included': 'Yes', 'microSD  up to 128 GB (uses SIM 2 slot)': 'Yes', 'microSD  up to 256 GB (dedicated slot) - single-SIM model (G930F  G930W8)': 'Yes', 'SD/MMC': 'Yes', 'SDIO/MMC + miniSD': 'No', 'miniSD  512 MB included': 'Yes', 'microSD  up to 2 GB (dedicated slot)  512 MB/ 2 GB included': 'Yes', 'Adreno 320': 'No', 'SD  up to 2 GB': 'Yes', 'microSD  up to 2 GB (dedicated slot)  512 MB included': 'Yes', 'Memory Stick Micro (M2)  up to 4 GB  1 GB included': 'No', 'microSD  up to 8 GB (dedicated slot)  128 MB included': 'Yes', 'Memory Stick Micro (M2)  up to 4 GB  256 MB included': 'No', 'Memory Stick Duo Pro  up to 2 GB  64 MB included': 'No', 'microSD  up to 16 GB': 'Yes', 'microSD  up to 512 MB (dedicated slot)': 'No', 'microSD  up to 64 GB (dedicated slot)  2 GB included': 'Yes', 'Memory Stick Micro (M2)  up to 8 GB  1 GB included': 'No', 'microSD  up to 256 GB': 'Yes', 'microSD  up to 1 GB (dedicated slot)': 'Yes', 'microSD  up to 8 GB (dedicated slot)  8 GB included': 'Yes', 'SD/MMC  up to 4 GB': 'Yes', 'Memory Stick Micro (M2)  up to 4 GB. 512 MB included': 'No', 'Memory Stick Micro (M2)  up to 2 GB': 'No', 'microSD  up to 32 GB (dedicated slot)  512 MB  included': 'Yes', 'microSD  up to 32 GB (dedicated slot)  4 GB card included': 'Yes', 'Memory Stick Micro (M2)/microSD  up to 4 GB (dedicated slot)  512 MB included': 'No', 'microSD  up to 16 GB (dedicated slot)  2 GB included': 'Yes', 'Memory Stick Micro (M2)  up to 4 GB  512 MB included': 'No', 'microSD  up to 8 GB (dedicated slot)  4 GB included': 'Yes', 'microSD  up to 8 GB (dedicated slot)': 'Yes', 'microSD  up to 4 GB (dedicated slot)  2 GB included': 'Yes', 'Memory Stick Micro (M2)  256 MB included': 'No', 'microSD  up to 8 GB (dedicated slot)  512 MB included': 'Yes', 'microSD  up to 256 GB (dedicated slot) - F5121': 'Yes', 'microSD  up to 4 GB (dedicated slot)': 'Yes', 'microSD  up to 256 GB (dedicated slot) - single SIM': 'Yes', 'MMC': 'Yes', 'Memory Stick Micro (M2)  up to  2 GB  256 MB included': 'No', 'Memory Stick Duo Pro  up to 2 GB  512 MB included': 'No', 'microSD  up to 128 GB': 'Yes', 'microSD (dedicated slot)  512 MB included': 'Yes', 'SD  up to 32 GB': 'Yes', 'Memory Stick Duo Pro  64 MB included': 'No', 'microSD  up to 16 GB (dedicated slot)': 'Yes', 'microSD  up to 256 GB (dedicated slot) - single & dual SIM models': 'Yes', 'Memory Stick Micro (M2)  up to 8 GB  2 GB included': 'No', 'microSD  up to 32 GB (dedicated slot)': 'Yes', 'microSD  up to 32 GB (dedicated slot)  2 GB included': 'Yes', 'microSD (dedicated slot)  up to 256 GB (LTE model)  32 GB (3G model)': 'Yes', 'Memory Stick Micro (M2)  up to 4 GB': 'No', 'miniSD  up to 16 GB': 'Yes', 'microSD  up to 256 GB (uses SIM 2 slot)': 'Yes', 'Memory Stick Duo Pro  up to 4 GB': 'No', 'microSD (dedicated slot)  16 GB included': 'Yes', 'microSD  up to 256 GB (uses SIM 1 slot)': 'Yes', 'microSD  up to 256 GB (dedicated slot) - F620S': 'Yes', 'microSD  up to 2 GB (dedicated slot)': 'Yes', 'microSD  up to 16 GB (dedicated slot)  8 GB included': 'Yes', 'SDIO': 'No', 'microSD  up to 8 GB (dedicated slot)  1 GB included': 'Yes', 'microSD (dedicated slot)  SD 2.0 compatible': 'Yes', 'microSD  up to 8 GB (dedicated slot)  2 GB included': 'Yes', 'microSD  up to 32 GB (dedicated slot)  2/4 GB included': 'Yes', 'MiniSD': 'No', 'microSD  up to 256 GB (dedicated slot)': 'Yes', 'microSD  up to 256GB (uses SIM2 slot)': 'Yes', 'microSD (dedicated slot)  2 GB included': 'Yes', 'microSD  up to 32 GB (dedicated slot) - Chinese version only': 'Yes', 'SD/MMC  up to 2 GB': 'Yes', 'microSD  up to 64 GB (dedicated slot)  16 GB included': 'Yes', 'microSD  up to 32 GB (dedicated slot) - market dependent': 'Yes', 'miniSD  up to 2 GB  128 MB included': 'No', 'microSD  up to 32 GB': 'Yes', 'RS-DV-MMC  128 MB included': 'Yes', 'microSD  up to 256 GB (dedicated slot) - G8231': 'Yes', 'microSD  up to 32 GB (dedicated slot)  32 GB included': 'Yes', 'Memory Stick Duo Pro  up to 4 GB  512 MB included': 'No', 'microSD': 'Yes', 'SD/SDIO': 'No', 'Memory Stick Micro (M2)  up to 8 GB': 'No', 'Memory Stick Duo Pro  up to 4 GB  256 MB included': 'No', 'microSD  up to 32 GB (dedicated slot)  16 GB included': 'Yes', 'Memory Stick Micro (M2)  up to 8 GB  4 GB included': 'No', 'Yes  up to 64 GB (dedicated slot)': 'Yes', 'microSD  up to 256 GB (dedicated slot) - non microSDXC compliant': 'Yes', 'Memory Stick Micro (M2)  up to 8 GB  2 GB/ 8 GB included': 'No', 'microSD  up to 32 GB (dedicated slot); SD cards': 'Yes', 'microSD   up to 512 MB (dedicated slot)': 'Yes', 'microSD  up to 16 GB (dedicated slot)  4 GB included': 'Yes', 'microSD  up to 200 GB (dedicated slot)': 'Yes', 'Memory Stick Micro (M2)   up to 8 GB  512 MB included': 'No', 'RS-MMC  32 MB included': 'Yes', 'Memory Stick Micro (M2)  up to 2 GB  256 MB included': 'No', 'microSD  up to 32 GB (uses SIM 2 slot) - mediatek model': 'Yes', 'Memory Stick Duo Pro  1 GB included': 'No', 'miniSDIO  128 MB included': 'No', 'microSD  up to 256 GB (dedicated slot) - single-SIM model - G935F  G935W8': 'Yes', 'microSD  up to 2 GB (dedicated slot)  64 MB included': 'Yes'}})

In [85]:
Phones3 = Phones2.replace({'2G': {'GSM / CDMA': 'Yes', 'GSM / UMTS / HSPA': 'Yes', 'GSM / HSPA / CDMA2000': 'Yes', 'LTE': 'No', 'CDMA / LTE': 'No', 'CDMA / CDMA2000': 'No', 'GSM / UMTS': 'Yes', 'HSPA / LTE': 'No', 'GSM / HSPA / EVDO / LTE': 'Yes', 'GSM / LTE': 'Yes', 'CDMA / EVDO': 'No', 'GSM / CDMA / EVDO / LTE': 'Yes', 'CDMA / EVDO / LTE': 'No', 'GSM / CDMA / HSPA / EVDO / LTE': 'Yes', 'GSM / HSPA / CDMA2000 / LTE': 'Yes', 'No cellular connectivity': 'No', 'HSPA / EVDO': 'No', 'GSM / CDMA / HSPA': 'Yes', 'CDMA / HSPA': 'No', 'GSM / CDMA2000': 'Yes', 'GSM / CDMA / HSPA / EVDO': 'Yes', 'GSM':'Yes', 'GSM / CDMA / HSPA / CDMA2000 / LTE': 'Yes', 'CDMA / HSPA / EVDO': 'No', 'CDMA': 'No', 'GSM / CDMA / EVDO': 'Yes', 'GSM / HSPA / EVDO': 'Yes', 'GSM / HSPA': 'Yes', 'CDMA / HSPA / EVDO / LTE': 'No', 'GSM / CDMA / UMTS / EVDO': 'Yes', 'GSM / CDMA / HSPA / LTE': 'Yes', 'GSM / UMTS / HSPA / LTE': 'Yes', 'CDMA / HSPA / LTE': 'No', 'HSPA': 'No', 'GSM / HSPA / LTE': 'Yes'}})

In [86]:
Phones4 = Phones3.replace({'3G': {'GSM / CDMA': 'Yes', 'GSM / UMTS / HSPA': 'Yes', 'GSM / HSPA / CDMA2000': 'Yes', 'LTE': 'No', 'CDMA / LTE': 'Yes', 'CDMA / CDMA2000': 'Yes', 'GSM / UMTS': 'Yes', 'HSPA / LTE': 'Yes', 'GSM / HSPA / EVDO / LTE': 'Yes', 'GSM / LTE': 'No', 'CDMA / EVDO': 'Yes', 'GSM / CDMA / EVDO / LTE': 'Yes', 'CDMA / EVDO / LTE': 'Yes', 'GSM / CDMA / HSPA / EVDO / LTE': 'Yes', 'GSM / HSPA / CDMA2000 / LTE': 'Yes', 'No cellular connectivity': 'No', 'HSPA / EVDO': 'Yes', 'GSM / CDMA / HSPA': 'Yes', 'CDMA / HSPA': 'Yes', 'GSM / CDMA2000': 'Yes', 'GSM / CDMA / HSPA / EVDO': 'Yes', 'GSM':'No', 'GSM / CDMA / HSPA / CDMA2000 / LTE': 'Yes', 'CDMA / HSPA / EVDO': 'Yes', 'CDMA': 'Yes', 'GSM / CDMA / EVDO': 'Yes', 'GSM / HSPA / EVDO': 'Yes', 'GSM / HSPA': 'Yes', 'CDMA / HSPA / EVDO / LTE': 'No', 'GSM / CDMA / UMTS / EVDO': 'Yes', 'GSM / CDMA / HSPA / LTE': 'Yes', 'GSM / UMTS / HSPA / LTE': 'Yes', 'CDMA / HSPA / LTE': 'Yes', 'HSPA': 'Yes', 'GSM / HSPA / LTE': 'Yes'}})

In [87]:
Phones5 = Phones4.replace({'4G': {'GSM / CDMA': 'No', 'GSM / UMTS / HSPA': 'No', 'GSM / HSPA / CDMA2000': 'No', 'LTE': 'Yes', 'CDMA / LTE': 'Yes', 'CDMA / CDMA2000': 'No', 'GSM / UMTS': 'No', 'HSPA / LTE': 'Yes', 'GSM / HSPA / EVDO / LTE': 'Yes', 'GSM / LTE': 'Yes', 'CDMA / EVDO': 'No', 'GSM / CDMA / EVDO / LTE': 'Yes', 'CDMA / EVDO / LTE':'Yes','GSM / CDMA / HSPA / EVDO / LTE': 'Yes', 'GSM / HSPA / CDMA2000 / LTE': 'Yes', 'No cellular connectivity': 'No', 'HSPA / EVDO': 'No', 'GSM / CDMA / HSPA': 'No', 'CDMA / HSPA': 'No', 'GSM / CDMA2000': 'No', 'GSM / CDMA / HSPA / EVDO': 'No', 'GSM':'No', 'GSM / CDMA / HSPA / CDMA2000 / LTE': 'Yes', 'CDMA / HSPA / EVDO': 'No', 'CDMA': 'No', 'GSM / CDMA / EVDO': 'No', 'GSM / HSPA / EVDO': 'No', 'GSM / HSPA': 'No', 'CDMA / HSPA / EVDO / LTE': 'Yes', 'GSM / CDMA / UMTS / EVDO': 'No', 'GSM / CDMA / HSPA / LTE': 'Yes', 'GSM / UMTS / HSPA / LTE': 'Yes', 'CDMA / HSPA / LTE': 'Yes', 'HSPA': 'No', 'GSM / HSPA / LTE': 'Yes'}})

In [88]:
Phones5['memory_card'].describe()

count     5004
unique       2
top        Yes
freq      4138
Name: memory_card, dtype: object

In [89]:
Phones5['2G'].describe()

count     5006
unique       2
top        Yes
freq      4586
Name: 2G, dtype: object

In [90]:
Phones5['3G'].describe()

count     5006
unique       2
top        Yes
freq      3484
Name: 3G, dtype: object

In [91]:
Phones5['4G'].describe()

count     5006
unique       2
top         No
freq      3711
Name: 4G, dtype: object

In [92]:
b = {}
b = set(Phones5['brand'])
print(b)

{nan, 'Archos', 'Yezz', 'Google', 'Motorola', 'Sagem', 'VK Mobile', 'BLU', 'HP', 'Spice', 'Micromax', 'i-mate', 'Microsoft', 'Toshiba', 'XOLO', 'Vertu', 'vivo', 'Mitac', 'BQ', 'Asus', 'Amazon', 'Gionee', 'Nvidia', 'BlackBerry', 'Sonim', 'Lenovo', 'Prestigio', 'Gigabyte', 'Sharp', 'LG', 'Lava', 'OnePlus', 'Meizu', 'Philips', 'Allview', 'BenQ', 'Celkon', 'Oppo', 'T-Mobile', 'Palm', 'Orange', 'Kyocera', 'Dell', 'HTC', 'QMobile', 'alcatel', 'Vodafone', 'Pantech', 'Neonode', 'Apple', 'Jolla', 'Coolpad', 'NEC', 'Sony', 'YU', 'O2', 'Yota', 'Karbonn', 'Qtek', 'Eten', 'ZTE', 'Panasonic', 'Intex', 'verykool', 'Sony Ericsson', 'Garmin-Asus', 'Samsung', 'Icemobile', 'Nokia', 'Cat', 'NIU', 'Casio', 'Xiaomi', 'Acer', 'Huawei', 'LeEco', 'iNQ', 'Wiko'}


In [93]:
c = list(b)
Phones6 = Phones5
for i in range(0, len(c)):
    Phones6[c[i]] = Phones5['brand']
    Phones6[c[i]] = np.where(Phones5[c[i]] == c[i], 1, 0)

In [103]:
Phones6.head(20)

,brand,network_technology,GPRS,EDGE,dimentions,weight_g,SIM,display_resolution,OS,CPU,...,Cat,NIU,Casio,Xiaomi,Acer,Huawei,LeEco,iNQ,Wiko,Thickness
0,Acer,GSM / HSPA / LTE,Yes,Yes,0.37,260,Dual SIM (Micro-SIM/Nano-SIM),7.0 inches (~69.8% screen-to-body ratio),Android 6.0 (Marshmallow),Quad-core 1.3 GHz Cortex-A53,...,0,0,0,0,1,0,0,0,0,0.37
1,Acer,GSM / HSPA / LTE,Yes,Yes,0.33,169,Single SIM (Micro-SIM) or Dual SIM (Micro-SIM|...,5.5 inches (~71.7% screen-to-body ratio),Android 6.0 (Marshmallow),Octa-core 1.3 GHz Cortex-A53,...,0,0,0,0,1,0,0,0,0,0.33
2,Acer,GSM / HSPA / LTE,Yes,Yes,-,NaN,Single SIM (Micro-SIM) or Dual SIM (Micro-SIM|...,5.0 inches,Android 6.0 (Marshmallow),Quad-core 1.25 GHz Cortex-A53,...,0,0,0,0,1,0,0,0,0,-
3,Acer,No cellular connectivity,No,No,0.35,NaN,No,10.1 inches (~68.4% screen-to-body ratio),Android 6.0 (Marshmallow),Quad-core 1.3 GHz Cortex-A53,...,0,0,0,0,1,0,0,0,0,0.35
4,Acer,GSM / HSPA / LTE,Yes,Yes,0.33,166,Triple SIM (Micro-SIM),5.5 inches (~69.0% screen-to-body ratio),Android 5.1 (Lollipop),Octa-core 1.3 GHz Cortex-A53,...,0,0,0,0,1,0,0,0,0,0.33
5,Acer,GSM / HSPA / LTE,Yes,Yes,0.39,NaN,Single SIM (Micro-SIM) or Dual SIM (Micro-SIM|...,5.5 inches (~70.3% screen-to-body ratio),Android 6.0 (Marshmallow),Quad-core 1.3 GHz Cortex-A53,...,0,0,0,0,1,0,0,0,0,0.39
6,Acer,GSM / HSPA / LTE,Yes,Yes,0.33,125,Single SIM (Micro-SIM) or Dual SIM (Micro-SIM|...,5.0 inches (~66.4% screen-to-body ratio),Android 6.0 (Marshmallow),Quad-core 1.3 GHz - Z525,...,0,0,0,0,1,0,0,0,0,0.33
7,Acer,No cellular connectivity,No,No,0.34,353.8,No,8.0 inches (~67.1% screen-to-body ratio),Android 5.0 (Lollipop),Quad-core 1.6 GHz,...,0,0,0,0,1,0,0,0,0,0.34
8,Acer,GSM / HSPA / LTE,Yes,Yes,0.33,150,Dual SIM (Nano-SIM| dual stand-by),5.5 inches (~70.2% screen-to-body ratio),Microsoft Windows 10,Hexa-core (4x1.4 GHz Cortex-A53 & 2x1.8 GHz Co...,...,0,0,0,0,1,0,0,0,0,0.33
9,Acer,GSM / HSPA / LTE,Yes,Yes,0.38,142,Single SIM (Micro-SIM) or Dual SIM (Micro-SIM|...,4.5 inches (~61.7% screen-to-body ratio),Android 5.1 (Lollipop),Quad-core 1.1 GHz Cortex-A7,...,0,0,0,0,1,0,0,0,0,0.38


In [95]:
E = Phones6['dimentions']
print(E)

0           191.7 x 101 x 9.4 mm (7.55 x 3.98 x 0.37 in)
1          153.8 x 75.6 x 8.5 mm (6.06 x 2.98 x 0.33 in)
2                                                      -
3            259 x 167 x 8.9 mm (10.20 x 6.57 x 0.35 in)
4          153.3 x 78.8 x 8.5 mm (6.04 x 3.10 x 0.33 in)
5               154 x 77 x 10 mm (6.06 x 3.03 x 0.39 in)
6          145.7 x 71.2 x 8.4 mm (5.74 x 2.80 x 0.33 in)
7            217.9 x 127 x 8.6 mm (8.58 x 5.0 x 0.34 in)
8          156.5 x 75.9 x 8.4 mm (6.16 x 2.99 x 0.33 in)
9            136 x 66.5 x 9.6 mm (5.35 x 2.62 x 0.38 in)
10           136 x 66.5 x 9.6 mm (5.35 x 2.62 x 0.38 in)
11         156.3 x 77.5 x 8.9 mm (6.15 x 3.05 x 0.35 in)
12           144 x 70.3 x 8.9 mm (5.67 x 2.77 x 0.35 in)
13           260 x 176 x 9.7 mm (10.24 x 6.93 x 0.38 in)
14        213.9 x 127.7 x 9.5 mm (8.42 x 5.03 x 0.37 in)
15      256.5 x 170.2 x 10.2 mm (10.10 x 6.70 x 0.40 in)
16      256.5 x 170.2 x 10.2 mm (10.10 x 6.70 x 0.40 in)
17           143.5 x 69.8 x 8 m

In [98]:
for i in range (0, len(E)):
    if (len(re.findall('\d+\.\d+', E[i]))>0):
        E[i] = (re.findall('\d+\.\d+', E[i]))[-1]

In [102]:
Phones6['Thickness'] = E

0       0.37
1       0.33
2          -
3       0.35
4       0.33
5       0.39
6       0.33
7       0.34
8       0.33
9       0.38
10      0.38
11      0.35
12      0.35
13      0.38
14      0.37
15      0.40
16      0.40
17      0.31
18      0.25
19      0.38
20      0.38
21      0.39
22      0.31
23      0.35
24      0.33
25      0.30
26      0.39
27      0.37
28      0.41
29      0.33
        ... 
4977    0.47
4978    0.47
4979    0.50
4980    0.55
4981    0.30
4982    0.38
4983    0.34
4984    0.39
4985    0.39
4986    0.44
4987    0.39
4988    0.40
4989    0.45
4990    0.47
4991    0.43
4992    0.52
4993    0.50
4994    0.58
4995    0.41
4996    0.49
4997    0.51
4998    0.50
4999    0.41
5000    0.46
5001    0.53
5002       -
5003    0.46
5004    0.50
5005    0.57
5006       -
Name: dimentions, Length: 5007, dtype: object


In [109]:
Phones7 = Phones6.drop(['brand', 'network_technology', 'dimentions'], axis = 1)

In [110]:
Phones7.to_csv("My_Refined.csv", index=False)